# Train a Model with SageMaker AutoPilot
We will use AutoPilot to predict sentiment of customer reviews.

# Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  AutoPilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and sentiment classification machine learning pipeline.

# Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Dataset

In [2]:
%store -r header_train_s3_uri

print(header_train_s3_uri)

s3://sagemaker-us-west-2-086401037028/data/amazon_reviews_us_Digital_Software_v1_00_header.csv


In [3]:
!aws s3 ls $header_train_s3_uri

2020-04-25 15:48:24   15164605 amazon_reviews_us_Digital_Software_v1_00_header.csv


# Setup the S3 Location for the AutoPilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [4]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_model_output)

print(model_output_s3_uri)


s3://sagemaker-us-west-2-086401037028/models/autopilot


In [5]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Launch the SageMaker AutoPilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [6]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

AutoMLJobName: automl-dm-25-15-49-25


_Note that we are not specifying the `ProblemType`.  AutoPilot will automatically detect if we're using regression or classification (binary or multi-class)._

In [7]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType="Classification",
                      RoleArn=role)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:086401037028:automl-job/automl-dm-25-15-49-25',
 'ResponseMetadata': {'RequestId': 'd50ffca0-91e1-444e-8310-1214c2482f65',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd50ffca0-91e1-444e-8310-1214c2482f65',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sat, 25 Apr 2020 15:49:24 GMT'},
  'RetryAttempts': 0}}

# Tracking the progress of the AutoPilot job
SageMaker AutoPilot job consists of the following high-level steps: 
* _Data Analysis_ where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* _Feature Engineering_ where the data is scrubbed, balanced, combined, and split into train and validation
* _Model Training and Tuning_ where the top performing features, hyper-parameters, and models are selected and trained.

# Analyzing Data

In [8]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
Data analysis complete
{'AutoMLJobName': 'automl-dm-25-15-49-25', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:086401037028:automl-job/automl-dm-25-15-49-25', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-086401037028/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-086401037028/models/autopilot'}, 'RoleArn': 'arn:aws:iam::086401037028:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTraini

In [9]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

{'AutoMLJobName': 'automl-dm-25-15-49-25', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:086401037028:automl-job/automl-dm-25-15-49-25', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-086401037028/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-086401037028/models/autopilot'}, 'RoleArn': 'arn:aws:iam::086401037028:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 4, 25, 15, 49, 25, 442000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 4, 25, 15, 56, 54, 433000, tzinfo=tzlocal()), 'AutoMLJobStatus': 'InProgress', 'AutoMLJobSecondaryStatus': 'FeatureEngineering', 'GenerateCandidateDefinitionsOnly': False, 'AutoMLJobArtifacts': {'CandidateDefini

# Feature Engineering

In [10]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress ModelTuning
Feature engineering complete
{'AutoMLJobName': 'automl-dm-25-15-49-25', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:086401037028:automl-job/automl-dm-25-15-49-25', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-086401037028/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-086401037028/models/autopilot'}, 'RoleArn': 'arn:aws:iam::086401037028:

# Model Training and Tuning

In [11]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
Completed MaxCandidatesReached
Model tuning complete
{'AutoMLJobName': 'automl-dm-25-15-49-25', 'AutoMLJobArn': 'arn:aws:sagemaker:us-west-2:086401037028:automl-job/automl-dm-25-15-49-25', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-west-2-086401037028/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-west-2-086401037028/models/autopilot'}, 'RoleArn': 'arn:aws:iam::086401037028:role/TeamRole', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 4, 25, 15, 49, 25, 442000, tzinfo=tzlocal()), 'EndTime': datetim

# Wait Until All Jobs are Done Above Before Proceeding

# View Generated Notebooks
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

## Copy the Generated Notebooks Locally

In [12]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

's3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/sagemaker-automl-candidates/pr-1-e140f6b397f3491399909c5c237e8704cc655a48014f4754b88b127e21'

In [13]:
!aws s3 cp --recursive $generated_resources .

download: s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/sagemaker-automl-candidates/pr-1-e140f6b397f3491399909c5c237e8704cc655a48014f4754b88b127e21/generated_module/MANIFEST.in to generated_module/MANIFEST.in
download: s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/sagemaker-automl-candidates/pr-1-e140f6b397f3491399909c5c237e8704cc655a48014f4754b88b127e21/generated_module/candidate_data_processors/dpp1.py to generated_module/candidate_data_processors/dpp1.py
download: s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/sagemaker-automl-candidates/pr-1-e140f6b397f3491399909c5c237e8704cc655a48014f4754b88b127e21/generated_module/candidate_data_processors/trainer.py to generated_module/candidate_data_processors/trainer.py
download: s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/sagemaker-automl-candidates/pr-1-e140f6b397f3491399909c5c237e8704cc655a48014f4754b88b127e21/generat

## In the file view, open the following folders:
```
notebooks/
generated_module/
```

Lots of useful information ^^ in these folders ^^

# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [14]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-7738074885aa48bc84-003-46cdd5f9  0.41137999296188354
1  tuning-job-1-7738074885aa48bc84-002-1dcaabb9  0.38307398557662964
2  tuning-job-1-7738074885aa48bc84-001-747d7472  0.2882860004901886


# Inspect Trials using Experiments API
SageMaker AutoPilot automatically creates a new experiment, and pushes information for each trial. 

In [15]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

                                  TrialComponentName  \
0  tuning-job-1-7738074885aa48bc84-003-46cdd5f9-a...   
1  tuning-job-1-7738074885aa48bc84-001-747d7472-a...   
2  tuning-job-1-7738074885aa48bc84-002-1dcaabb9-a...   
3  automl-dm--dpp1-csv-1-5e565e32dcc84e09a1e3d9d8...   
4  automl-dm--dpp0-rpb-1-ce3801a07cc04e9f93315271...   
5  automl-dm--dpp0-1-afe8079debca490e892a165cc21e...   
6  automl-dm--dpp1-1-89237334dae2409e96dd6cacbb4d...   
7  db-1-358c8cffe08746ffbb2e3c8bf60bc54f4c40d9e1a...   

                                         DisplayName  \
0  tuning-job-1-7738074885aa48bc84-003-46cdd5f9-a...   
1  tuning-job-1-7738074885aa48bc84-001-747d7472-a...   
2  tuning-job-1-7738074885aa48bc84-002-1dcaabb9-a...   
3  automl-dm--dpp1-csv-1-5e565e32dcc84e09a1e3d9d8...   
4  automl-dm--dpp0-rpb-1-ce3801a07cc04e9f93315271...   
5  automl-dm--dpp0-1-afe8079debca490e892a165cc21e...   
6  automl-dm--dpp1-1-89237334dae2409e96dd6cacbb4d...   
7  db-1-358c8cffe08746ffbb2e3c8bf60bc54f4c40d9e

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [16]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: tuning-job-1-7738074885aa48bc84-003-46cdd5f9
Metric name: validation:accuracy
Metric value: 0.41137999296188354


In [17]:
best_candidate

{'CandidateName': 'tuning-job-1-7738074885aa48bc84-003-46cdd5f9',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.41137999296188354},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:086401037028:processing-job/db-1-358c8cffe08746ffbb2e3c8bf60bc54f4c40d9e1a4ef44578d79d0420c',
   'CandidateStepName': 'db-1-358c8cffe08746ffbb2e3c8bf60bc54f4c40d9e1a4ef44578d79d0420c'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:086401037028:training-job/automl-dm--dpp0-1-afe8079debca490e892a165cc21ef61c3f75402a26b74',
   'CandidateStepName': 'automl-dm--dpp0-1-afe8079debca490e892a165cc21ef61c3f75402a26b74'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-west-2:086401037028:transform-job/automl-dm--dpp0-rpb-1-ce3801a07cc04e9f93315271805bc28d191

We can see the containers and models composing the Inference Pipeline.

In [18]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/data-processor-models/automl-dm--dpp0-1-afe8079debca490e892a165cc21ef61c3f75402a26b74/output/model.tar.gz
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3
s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/tuning/automl-dm--dpp0-xgb/tuning-job-1-7738074885aa48bc84-003-46cdd5f9/output/model.tar.gz
246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-west-2-086401037028/models/autopilot/automl-dm-25-15-49-25/data-processor-models/automl-dm--dpp0-1-afe8079debca490e892a165cc21ef61c3f75402a26b74/output/model.tar.gz


# AutoPilot Chooses XGBoost as Best Candidate!
Note that AutoPilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [19]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Best candidate model ARN:  arn:aws:sagemaker:us-west-2:086401037028:model/automl-dm-model-25-15-49-25


In [20]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
xgb_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(xgb_endpoint_name)
print(variant_name)

automl-dm-ep-25-16-08-34
automl-dm-variant-25-16-08-34


In [21]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m4.xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [22]:
create_endpoint_response = sm.create_endpoint(EndpointName=xgb_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-west-2:086401037028:endpoint/automl-dm-ep-25-16-08-34


# Wait for the Model to Deploy
This may take 5-10 mins.  Please be patient.

In [23]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=xgb_endpoint_name)


In [24]:
resp = sm.describe_endpoint(EndpointName=xgb_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-west-2:086401037028:endpoint/automl-dm-ep-25-16-08-34
Status: InService


# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [25]:
sm_runtime = boto3.client('sagemaker-runtime')

In [26]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'4'

In [27]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'1'

In [28]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'3'

# Summary
We used AutoPilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

AutoPilot uses a white-box approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the AutoPilot service.